This notebook uses the OpenStudio Meta-CLI to submit OSA jobs to OpenStudio-Server.  

The OpenStudio Meta-CLI is a ruby executable that can parse OpenStudio Analysis (OSA) json files and submit the appropriate job type and payload to a running OpenStudio-Server. These files can describe complex analyses, such as building-stock analysis, design optimization, uncertainty quantification, and calibration.  This example requires a running server.

In [1]:
RUBY_CMD = 'C:\ParametricAnalysisTool-3.3.0\pat\ruby\bin\ruby'
#META_CLI = File.absolute_path('/opt/OpenStudio-server/bin/openstudio_meta')
META_CLI = File.absolute_path('C:\ParametricAnalysisTool-3.3.0\pat\OpenStudio-server\bin\openstudio_meta')
PROJECT = Dir.pwd
HOST = 'localhost'

@host = HOST
@project = PROJECT
@meta_cli = META_CLI
@ruby_cmd = RUBY_CMD

"C:\\ParametricAnalysisTool-3.3.0\\pat\\ruby\\bin\\ruby"

In [2]:
command = "#{@ruby_cmd} #{@meta_cli} run_analysis --debug --verbose '#{@project}/analysis.json' 'http://#{@host}' -z 'analysis' -a lhs"

"C:\\ParametricAnalysisTool-3.3.0\\pat\\ruby\\bin\\ruby C:/ParametricAnalysisTool-3.3.0/pat/OpenStudio-server/bin/openstudio_meta run_analysis --debug --verbose 'C:/Projects/Notebooks/osw_project/analysis.json' 'http://localhost' -z 'analysis' -a lhs"

In [3]:
run_analysis = system(command)

true

In [4]:
sleep 5

5

In [5]:
require 'rest-client'
require 'json'
a = RestClient.get "http://#{@host}/analyses.json"
a = JSON.parse(a, symbolize_names: true)
a = a.sort { |x, y| x[:created_at] <=> y[:created_at] }.reverse
analysis = a[0]
analysis_id = analysis[:_id]

"6d92a71c-230d-4212-85a1-7dbd2753ce67"

In [6]:
a = RestClient.get "http://#{@host}/analyses/#{analysis_id}/status.json"
a = JSON.parse(a, symbolize_names: true)
status = a[:analysis][:status]

"queued"

In [7]:
a = RestClient.get "http://#{@host}/analyses/#{analysis_id}.json"
a = JSON.parse(a, symbolize_names: true)
status = a[:analysis][:problem][:analysis_type]

"lhs"

In [8]:
# get all data points in this analysis
a = RestClient.get "http://#{@host}/data_points.json"
a = JSON.parse(a, symbolize_names: true)
data_points = []
a.each do |data_point|
  if data_point[:analysis_id] == analysis_id
    data_points << data_point
  end
end
data_points.size

10

In [9]:
data_point_id = data_points[0][:_id]

"27543079-5f0b-44f7-b0b6-8527f2720d6f"

In [10]:
dp = RestClient.get "http://#{@host}/data_points/#{data_point_id}.json"
dp = JSON.parse(dp, symbolize_names: true)
data_points_status = dp[:data_point][:status]

"queued"

In [11]:
dp[:data_point][:status_message]

""

In [12]:
results = dp[:data_point][:results][:calibration_reports_enhanced]

In [13]:
results.slice(:electricity_consumption_cvrmse, :electricity_consumption_nmbe, :natural_gas_consumption_cvrmse, :natural_gas_consumption_nmbe)

NoMethodError: undefined method `slice' for nil:NilClass

In [14]:
objectives_json = RestClient.get "http://#{@host}/data_points/#{data_point_id}/download_result_file?filename=objectives.json"
objectives_json = JSON.parse(objectives_json, symbolize_names: true)

RestClient::UnprocessableEntity: 422 Unprocessable Entity

In [15]:
require 'open-uri'
File.write 'results.csv', URI.open("http://#{@host}/analyses/#{analysis_id}/download_data.csv?export=true").read


1364